## 1. Basic usage

### 1.1 Lazy Compilation

In this mode, compilation will be deferred until the first function execution. Numba will infer the argument types at call time, and generate optimized code based on this information. Numba will also be able to compile separate specializations depending on the input types. For example, calling the `f()` function above with integer or complex numbers will generate different code paths:

In [12]:
from numba import jit

@jit
def f(x, y):  # when input types change, the compilation will be redone.
    return x + y

def g(x, y):
    return x + y

In [14]:
%timeit -r 10 -n 5 f(1, 2)  # first run 1.44ms because of compilation

332 ns ± 189 ns per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [16]:
%timeit -r 10 -n 5 f("1", "2")  # will recompilation because of input type's changing.

10.4 µs ± 2.55 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


### 1.2 Eager compilation

In [17]:
from numba import jit, int32

@jit(int32(int32, int32))
def f(x, y):
    return x + y

In [20]:
%timeit -r 10 -n 5 f(1, 2)  # first run never cost more time on compilation

350 ns ± 215 ns per loop (mean ± std. dev. of 10 runs, 5 loops each)


`int32(int32, int32)` is the function’s signature. In this case, the corresponding specialization will be compiled by the `@jit` decorator, and no other specialization will be allowed. This is useful if you want fine-grained control over types chosen by the compiler (for example, to use single-precision floats).

If you omit the return type, e.g. by writing `(int32, int32)` instead of `int32(int32, int32)`, Numba will try to infer it for you. Function signatures can also be strings, and you can pass several of them as a list; 

In [22]:
f("1", "2")  # The type unmatched, will raise error

TypeError: No matching definition for argument type(s) str, str

In [24]:
f(2**31, 2**31)  # The result is also reasonable in `int32` range.

0

## 2. Calling and inlining other functions

Numba-compiled functions can call other compiled functions. The function calls may even be inlined in the native code, depending on optimizer heuristics. For example:

In [32]:
import math

@jit
def square(x):
    return x ** 2

@jit
def hypot(x, y):  # Both @jit
    return math.sqrt(square(x) + square(y))

def square2(x):
    return x ** 2

def hypot2(x, y):  # Neither @jit
    return math.sqrt(square2(x) + square2(y))

@jit
def hypot3(x, y):  # Half @jit
    return math.sqrt(square2(x) + square2(y))

In [34]:
%timeit -r 10 -n 5 hypot(10, 20)

332 ns ± 191 ns per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [35]:
%timeit -r 10 -n 5 hypot2(10, 20)

1.12 µs ± 249 ns per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [37]:
%timeit -r 10 -n 5 hypot3(10, 20)  # Half decorated is slower than neither.

2.73 µs ± 1.25 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


## 3. Signature specifications

Explicit `@jit` signatures can use a number of types. Here are some common ones:  

* `void` is the return type of functions returning nothing (which actually return `None` when called from Python).
* `intp` and `uintp` are pointer-sized integers (signed and unsigned, respectively)
* `intc` and `uintc` are equivalent to `C int` and `unsigned int` integer types
* `int8`, `uint8`, `int16`, `uint16`, `int32`, `uint32`, `int64`, `uint64` are fixed-width integers of the corresponding bit width (signed and unsigned)
* `float32` and `float64` are single- and double-precision floating-point numbers, respectively  
* `complex64` and `complex128` are single- and double-precision complex numbers, respectively
* array types can be specified by indexing any numeric type, e.g. `float32[:]` for a one-dimensional single-precision array or `int8[:,:]` for a two-dimensional array of 8-bit integers.

## 4. Compilation options

### 4.1 <font color=#FF0000 >nopython</font>

Numba has two compilation modes: `nopython` mode and `object` mode. The former produces much faster code, but has limitations that can force Numba to fall back to the latter. To prevent Numba from falling back, and instead raise an error, pass `nopython=True`.

### 4.2 <font color=#FF0000 >nogil</font>

Whenever Numba optimizes Python code to native code that only works on native types and variables (rather than Python objects), it is not necessary anymore to hold Python’s *global interpreter lock (GIL)*. Numba will release the GIL when entering such a compiled function if you passed `nogil=True`.

Code running with the GIL released runs concurrently with other threads executing Python or Numba code (either the same compiled function, or another one), allowing you to take advantage of multi-core systems. This will not be possible if the function is compiled in object mode.  
  
When using `nogil=True`, *you’ll have to be wary of the usual pitfalls of multi-threaded programming (consistency, synchronization, race conditions, etc.).*

### 4.3 <font color=#FF0000 >cache</font>

To avoid compilation times each time you invoke a Python program, you can instruct Numba to write the result of function compilation into a file-based cache. This is done by passing `cache=True`:

### 4.4 <font color=#FF0000 >parallel</font>

Enables automatic parallelization (and related optimizations) for those operations in the function known to have parallel semantics. This feature is enabled by passing `parallel=True` and must be used in conjunction with `nopython=True`.